## Import libraries and dependencies


In [6]:
import fetch_data as fd
import yfinance as yf
import quantstats as qs

In [28]:
# Request crypto tickers from user, and record as list
ticker_list = fd.input_cryptos()

# Fetch data from yfinance for each ticker, and create pandas dataframe
portfolio_df = fd.get_ticker_data(ticker_list)

Please enter the tickers of your cryptocurrencies one by one
Type 'done' when finished.


Ticker:  XRP
Ticker:  XDC
Ticker:  ADA
Ticker:  HBAR
Ticker:  done


--------------------------
Crpytocurrencies selected:
XRP, XDC, ADA, HBAR
--------------------------



Please select the interval: 
(Valid intervals: 1d, 5d, 1wk, 1mo, 3mo)
 1d


In [30]:
portfolio_df.head()
qs.stats.sortino(portfolio_df['XDC-USD']['daily_return'])
qs.reports.metrics(portfolio_df['XDC-USD']['daily_return'])

                    Strategy
------------------  ----------
Start Period        2019-01-08
End Period          2022-01-07
Risk-Free Rate      0.0%
Time in Market      100.0%

Cumulative Return   11,640.36%
CAGR﹪              389.66%

Sharpe              1.41
Sortino             2.73
Sortino/√2          1.93
Omega               1.32

Max Drawdown        -68.77%
Longest DD Days     247

Gain/Pain Ratio     0.32
Gain/Pain (1M)      3.0

Payoff Ratio        1.51
Profit Factor       1.32
Common Sense Ratio  1.98
CPC Index           0.93
Tail Ratio          1.49
Outlier Win Ratio   4.41
Outlier Loss Ratio  3.55

MTD                 -7.95%
3M                  -32.32%
6M                  -5.94%
YTD                 -7.95%
1Y                  1129.85%
3Y (ann.)           389.66%
5Y (ann.)           389.66%
10Y (ann.)          389.66%
All-time (ann.)     389.66%

Avg. Drawdown       -33.12%
Avg. Drawdown Days  70
Recovery Factor     169.27
Ulcer Index         0.39
Serenity Index      44.48
